In [1]:
# install requirements
import sys
if 'google.colab' in sys.modules:
    print('Running in Colab.')
    !pip3 install transformers==4.25.1 timm==0.4.12 fairscale==0.4.4
    !git clone https://github.com/salesforce/BLIP
    %cd BLIP

Running in Colab.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.9/93.9 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.4/235.4 kB 19.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 86.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.0/377.0 kB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 108.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 79.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 63.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━

# SPIQA

In [2]:
!unzip /content/SPIQA_testA_Images.zip

Archive:  /content/SPIQA_testA_Images.zip
   creating: SPIQA_testA_Images/
   creating: SPIQA_testA_Images/1809.03449v3/
  inflating: SPIQA_testA_Images/1809.03449v3/1809.03449v3-Table3-1.png  
  inflating: SPIQA_testA_Images/1809.03449v3/1809.03449v3-Table1-1.png  
  inflating: SPIQA_testA_Images/1809.03449v3/1809.03449v3-Figure3-1.png  
  inflating: SPIQA_testA_Images/1809.03449v3/1809.03449v3-Figure1-1.png  
  inflating: SPIQA_testA_Images/1809.03449v3/1809.03449v3-Figure4-1.png  
  inflating: SPIQA_testA_Images/1809.03449v3/1809.03449v3-Figure2-1.png  
  inflating: SPIQA_testA_Images/1809.03449v3/1809.03449v3-Table2-1.png  
   creating: SPIQA_testA_Images/1710.01507v4/
  inflating: SPIQA_testA_Images/1710.01507v4/1710.01507v4-Table1-1.png  
  inflating: SPIQA_testA_Images/1710.01507v4/1710.01507v4-Figure1-1.png  
   creating: SPIQA_testA_Images/1709.02755v5/
  inflating: SPIQA_testA_Images/1709.02755v5/1709.02755v5-Table1-1.png  
  inflating: SPIQA_testA_Images/1709.02755v5/1709.02

In [3]:
from PIL import Image
import requests
import torch
from torchvision import transforms
from torchvision.transforms.functional import InterpolationMode
import pandas as pd
from tqdm import tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
df = pd.read_csv('/content/sample.csv')
df

,paper,question,answer,reference_figure,reference_figure_caption,generated_answer
0,1611.04684v1,What are the main differences between the educ...,The Bonaparte school focuses on outdoor physic...,1611.04684v1-Table1-1.png,A difficult example from QA,NaN
1,1611.04684v1,Which model performs the best for response sel...,The KEHNN model performs the best for response...,1611.04684v1-Table4-1.png,Table 4: Evaluation results on response selection,NaN
2,1611.04684v1,Which model performs best on the Ubuntu datase...,KEHNN,1611.04684v1-Table5-1.png,Accuracy on different length of text,NaN
3,1611.04684v1,What is the role of the knowledge gates in the...,The knowledge gates are responsible for select...,1611.04684v1-Figure1-1.png,Architecture of KEHNN,NaN
4,1611.04684v1,How does the average number of answers per que...,The training set has a higher average number o...,1611.04684v1-Table2-1.png,Table 2: Statistics of the answer selection da...,NaN
...,...,...,...,...,...,...
661,1809.04276v2,How does the discriminator in the proposed REA...,The discriminator takes as input a response an...,1809.04276v2-Figure1-1.png,Figure 1: An overview of our proposed approach...,NaN
662,1710.01507v4,What is the role of the LSTM network in the mo...,The LSTM network is used to process the post t...,1710.01507v4-Figure1-1.png,Model Architecture,NaN
663,1709.00139v4,Which method generally achieved a lower object...,"For all datasets presented, Incremental SVM ac...",1709.00139v4-Table1-1.png,Table 1: Experimental Results of FISVDD and In...,NaN
664,1809.01989v2,Which method achieved the highest tracking acc...,The Ridge method achieved the lowest sum of ab...,1809.01989v2-Table1-1.png,Table 1. Absolute percentage errors for differ...,NaN


In [5]:
def load_image_and_question(image_path, image_size=480):
    path = "/content/BLIP/SPIQA_testA_Images/" + image_path
    raw_image = Image.open(path).convert('RGB')
    display(raw_image)
    transform = transforms.Compose([
        transforms.Resize((image_size,image_size),interpolation=InterpolationMode.BICUBIC),
        transforms.ToTensor(),
        transforms.Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711))
        ])
    image = transform(raw_image).unsqueeze(0).to(device)
    return image

In [6]:
from models.blip_vqa import blip_vqa

model_url = 'https://storage.googleapis.com/sfr-vision-language-research/BLIP/models/model_base_vqa_capfilt_large.pth'

model = blip_vqa(pretrained=model_url, image_size=480, vit='base')
model.eval()
model = model.to(device)

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

100%|██████████| 1.35G/1.35G [01:01<00:00, 23.5MB/s]


load checkpoint from https://storage.googleapis.com/sfr-vision-language-research/BLIP/models/model_base_vqa_capfilt_large.pth


In [ ]:
answers = []
for i, row in df.iterrows():
    image_path = row['paper'] + '/' + row['reference_figure']
    # question = "Given: " + row['reference_figure_caption'].strip() + '\n' + row['question'].strip()
    question = row['question'].strip()
    image = load_image_and_question(image_path)
    print("Question:", question)
    print("Actual Answer:", row['answer'].strip())
    with torch.no_grad():
        answer = model(image, question, train=False, inference='generate')
        answers.append(answer[0])
    print("Answer:", answer[0])
    print()

In [ ]:
df['generated_answer'] = answers
df.to_csv("blip_results.csv", index=False)